In [1]:
import sys
!conda install --yes --prefix {sys.prefix} fastparquet

Retrieving notices: ...working... done
Channels:
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_parquet('task_3_events.parquet', engine='fastparquet')
df.sort_values(['user_id', 'time'], inplace=True)

In [4]:
df['diff'] = df.groupby('user_id')['time'].diff()
df['session_date'] = df['time'].dt.date
df['session_start']= df[(df['diff'] >= '0 days 00:30:00') | (df['diff'].isna())]['time'].dt.time
df['session_end']=df[(~df['session_start'].shift(-1).isna()) | (df['time'].shift(-1).isna())]['time'].dt.time
df

,user_id,user_group,time,diff,session_date,session_start,session_end
82843,test_user_10021823,1,2023-12-07 01:08:51.008309,NaT,2023-12-07,01:08:51.008309,NaN
82853,test_user_10021823,1,2023-12-07 01:08:53.085064,0 days 00:00:02.076755,2023-12-07,NaN,NaN
82865,test_user_10021823,1,2023-12-07 01:08:56.702185,0 days 00:00:03.617121,2023-12-07,NaN,NaN
82866,test_user_10021823,1,2023-12-07 01:08:57.730887,0 days 00:00:01.028702,2023-12-07,NaN,NaN
82867,test_user_10021823,1,2023-12-07 01:08:57.808475,0 days 00:00:00.077588,2023-12-07,NaN,NaN
...,...,...,...,...,...,...,...
253401,test_user_999580,0,2023-12-06 15:42:07.074273,0 days 00:00:02.303254,2023-12-06,NaN,15:42:07.074273
209051,test_user_9998026,1,2023-12-06 15:36:14.988054,NaT,2023-12-06,15:36:14.988054,NaN
209054,test_user_9998026,1,2023-12-06 15:36:15.099478,0 days 00:00:00.111424,2023-12-06,NaN,NaN
209056,test_user_9998026,1,2023-12-06 15:36:15.535345,0 days 00:00:00.435867,2023-12-06,NaN,NaN


In [5]:
df_sessions = df[(~df['session_start'].isna())].reset_index()[['user_id', 'user_group', 'session_date', 'session_start']].join(df[~df['session_end'].isna()].reset_index()[['session_end']])
df_sessions

,user_id,user_group,session_date,session_start,session_end
0,test_user_10021823,1,2023-12-07,01:08:51.008309,01:10:11.804765
1,test_user_10030144,0,2023-12-02,20:37:34.861127,20:38:14.301560
2,test_user_1003396,1,2023-12-01,00:00:48.238407,00:01:41.376842
3,test_user_1003396,1,2023-12-02,08:21:13.134305,08:35:03.351138
4,test_user_1003396,1,2023-12-02,21:12:09.512252,21:34:53.437896
...,...,...,...,...,...
25039,test_user_997142,0,2023-12-13,19:11:15.701582,19:35:07.340102
25040,test_user_997142,0,2023-12-13,20:44:51.855149,21:04:34.052487
25041,test_user_999580,0,2023-12-03,14:59:54.018927,15:06:14.859587
25042,test_user_999580,0,2023-12-06,15:41:35.592334,15:42:07.074273


In [40]:
def getConversion(df_input, user_group):
    converted = (df_input[df_input['user_group'] == user_group].groupby('user_id')['session_start'].count() >= 2).sum()
    total = df_input[df_input['user_group'] == user_group].groupby('user_id').count().shape[0]
    return converted, total, (converted / total)*100

In [41]:
getConversion(df_sessions, 0)

(1833, 7429, 24.673576524431283)

In [42]:
getConversion(df_sessions, 1)

(1850, 7399, 25.003378834977703)